In [1]:
import os
import fitz
from tqdm import tqdm
from shutil import copy
import zipfile
import pandas as pd
from shutil import move
from ExtractPdf import *
from Sinopac import Sinopac
from Ibf import Ibf
from Ctbc import Ctbc
from Fubon import Fubon
from Yuanta import Yuanta

In [2]:
def handler_test(directory_path, date, processed_path):
    count_false = 0
    count_true = 0
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            full_path = os.path.join(directory_path, filename)
            old_name = os.path.splitext(filename)[0] # get the original filename without extension      
            # Split the base name into its components
            components = old_name.split('_')
            # Extract the components
            stock_num = components[0]
            stock_name = components[1]
            advisor_name = components[2]
            old_rate = components[3]
            summary = 'NULL'
            pdf_handler = ExtractPdf()
            if '永豐' in advisor_name:
                new_rate = pdf_handler.sinopac(full_path)
            elif '國票' in advisor_name:
                new_rate = pdf_handler.ibf(full_path)
            elif 'CTBC' in advisor_name or '中信' in advisor_name:
                new_rate = pdf_handler.ctbc(full_path)
            elif '台新' in advisor_name:
                new_rate = pdf_handler.taishin(full_path)
            elif '宏遠' in advisor_name:
                new_rate = pdf_handler.honsec(full_path)
            elif '元大' in advisor_name:
                new_rate = pdf_handler.yuanta(full_path)
            elif '元富' in advisor_name:
                new_rate = pdf_handler.masterlink(full_path)
            elif '富邦' in advisor_name:
                new_rate = pdf_handler.fubon(full_path)
            elif '統一' in advisor_name:
                new_rate = pdf_handler.pscnet(full_path)
            elif '群益' in advisor_name or 'capital' in advisor_name:
                new_rate = pdf_handler.capital(full_path)
            elif '第一金' in advisor_name or '一銀' in advisor_name:
                new_rate = pdf_handler.ffhc(full_path)  
            elif '日昇' in advisor_name:
                new_rate = pdf_handler.jihsun(full_path) 
            elif '玉山' in advisor_name:
                new_rate = pdf_handler.esun(full_path) 
            elif '國泰' in advisor_name:
                new_rate = pdf_handler.cathay(full_path) 
            elif '兆豐' in advisor_name:
                new_rate = pdf_handler.mega(full_path) 
            elif '福邦' in advisor_name:
                new_rate = pdf_handler.gfortune(full_path)      
            elif '康和' in advisor_name:
                new_rate = pdf_handler.concords(full_path) 
            elif 'MS' in advisor_name or 'ms' in advisor_name:
                new_rate = pdf_handler.morganstanley(full_path)
            elif 'CLSA' in advisor_name or 'cl' in advisor_name:
                new_rate = pdf_handler.clsa(full_path)  
            elif 'citi' in advisor_name:
                new_rate = pdf_handler.citi(full_path) 
            elif 'GS' in advisor_name or '高盛' in advisor_name or 'gs' in advisor_name:
                new_rate = pdf_handler.goldmansachs(full_path)  
            elif 'hsbc' in advisor_name or 'HSBC' in advisor_name:
                new_rate = pdf_handler.hsbc(full_path)  
            elif 'jpm' in advisor_name or '摩根大通' in advisor_name:
                new_rate = pdf_handler.jpmorgan(full_path)  
            elif 'NMR' in advisor_name or '野村' in advisor_name:
                new_rate = pdf_handler.nomura(full_path)   
            else :
                # new_rate = 'NULL'
                new_rate = 'wrong'
            new_name = f'{stock_num}_{stock_name}_{date}_{advisor_name}_{new_rate}_{summary}.pdf'
            new_path = os.path.join(processed_path, new_name)
            # copy(full_path, new_path) # copy the processed file to the output directory
            if new_rate == 'NULL':
                # count_true += 1
                # print(f'filename = {filename}')
                # print(f'advisor_name={advisor_name}, new_rate={new_rate}')
                pass
            elif new_rate == 'wrong':
                count_true += 1
                print(f'filename = {filename}')
                print(f'advisor_name={advisor_name}, new_rate={new_rate}')
                # pass
            elif new_rate == old_rate:
                count_true += 1
            elif new_rate in old_rate:
                count_true += 1
            elif old_rate in new_rate:
                count_true += 1
            else:
                count_true += 1
                # print(f'filename = {filename}')
                # print(f'advisor_name={advisor_name}, new_rate={new_rate}')
    count_false = len(os.listdir(directory_path)) - count_true
    success_rate = round((count_true/len(os.listdir(directory_path))), 3)*100
    print(f'共{len(os.listdir(directory_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')


In [3]:
def handler_find_advisor(directory_path):
    count_false = 0
    count_true = 0
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            full_path = os.path.join(directory_path, filename)
            old_name = os.path.splitext(filename)[0] # get the original filename without extension      
            # Split the base name into its components
            # Extract the components
            advisor = 'NULL'
            pdf_handler = ExtractPdf()
            ADVISOR_COMPANIES = [pdf_handler.sinopac, pdf_handler.ibf, pdf_handler.ctbc, 
                                 pdf_handler.taishin, pdf_handler.honsec, pdf_handler.yuanta, 
                                 pdf_handler.masterlink, pdf_handler.fubon, pdf_handler.pscnet, 
                                 pdf_handler.capital, pdf_handler.ffhc, pdf_handler.jihsun, 
                                 pdf_handler.esun, pdf_handler.cathay, pdf_handler.mega, 
                                 pdf_handler.gfortune, pdf_handler.concords, pdf_handler.morganstanley,
                                 pdf_handler.clsa, pdf_handler.citi, pdf_handler.goldmansachs, 
                                 pdf_handler.hsbc, pdf_handler.jpmorgan, pdf_handler.nomura]
            for company in ADVISOR_COMPANIES:
                new_rate = company(full_path)
                if pdf_handler.advisor != 'NULL':
                    advisor = pdf_handler.advisor
                    break
            if advisor == 'NULL':
                # print(full_path)
                count_false+=1
    if len(os.listdir(directory_path))>0:
        count_true = len(os.listdir(directory_path)) - count_false
        success_rate = round((count_true/(len(os.listdir(directory_path)))), 3)*100
        print(f'{directory_path}, 共{len(os.listdir(directory_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')


In [4]:
result = pd.DataFrame(columns=['filename', 'advisor', 'rating_1', 'rating_2', 'rating'])

In [6]:
root = '../history_data/file/中信託/'
count_false_rating = 0
count_true_rating = 0
count_false_tp = 0
count_true_tp = 0
count_false_author = 0
count_true_arthor = 0
count_false_summary = 0
count_true_summary = 0

count = 0

for filename in os.listdir(root):
    file_path = os.path.join(root, filename)
    pdfReader = Ctbc(file_path)
    rating = pdfReader.get_rating_process()
    # target_price = pdfReader.get_target_price_process()
    # author = pdfReader.get_author_process()
    # summary = pdfReader.get_summary_process()

    if rating == 'NULL':
        print(filename)
        print(rating)
        count_false_rating+=1

    new_row = {'filename':filename, 'advisor':pdfReader.advisor, 'rating_1':pdfReader.rating_1, 'rating_2':pdfReader.author_2, 'rating':rating}
    result = pd.concat([result, pd.DataFrame(new_row, index=[0])], ignore_index=True)
    count += 1
    # if count>100:
    #     break
    # if target_price == 'NULL':
    #     count_false_tp+=1

    # if author == 'NULL':
    #     print(filename)
    #     print(rating)
    #     print(target_price)
    #     print('author_1', pdfReader.author_1)
    #     print('author_2', pdfReader.author_2)
    #     print(summary)
    #     count_false_author+=1

    # if summary == 'NULL':
    #     count_false_summary+=1

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_rating
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_rating}件, 成功率:{success_rate}%')

# if len(os.listdir(root))>0:
#     count_true = len(os.listdir(root)) - count_false_tp
#     success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
#     print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_tp}件, 成功率:{success_rate}%')

# if len(os.listdir(root))>0:
#     count_true = len(os.listdir(root)) - count_false_author
#     success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
#     print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_author}件, 成功率:{success_rate}%')

# if len(os.listdir(root))>0:
#     count_true = len(os.listdir(root)) - count_false_summary
#     success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
#     print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_summary}件, 成功率:{success_rate}%')


1101_台泥_2018-05-17_中信投顧_中信個股報告-20180517_1101台泥.pdf
NULL
1101_台泥_2018-11-20_中信投顧_中信個股報告-20181120_1101台泥.pdf
NULL
1101_台泥_2019-05-14_中信投顧_中信個股報告-20190514_1101台泥.pdf
NULL
1102_亞泥_2018-04-26_中信投顧_中信個股報告-20180426_1102亞泥.pdf
NULL
1102_亞泥_2018-05-17_中信投顧_中信個股報告-20180517_1102亞泥.pdf
NULL
1102_亞泥_2018-12-03_中信投顧_中信個股報告-20181203_1102亞泥.pdf
NULL
1102_亞泥_2019-04-24_中信投顧_中信個股報告-20190424_1102亞泥.pdf
NULL
1203_味王_2018-12-10_中信投顧_中信個股報告-20181210_1203味王.pdf
NULL
1210-大成-2019_06_20-NULL-NULL.pdf
NULL
1210-大成-2019_06_20-中信投顧-NULL.pdf
NULL
1210_大成_2018-07-31_中信投顧_中信個股報告-20180731_1210大成.pdf
NULL
1210_大成_2019-01-11_中信投顧_中信個股報告-20190111_1210大成.pdf
NULL
1210_大成_2019-06-20__中信個股報告-20190620_1210大成.pdf
NULL
1215_卜蜂_2018-12-17_中信投顧_中信個股報告-20181217_1215卜蜂.pdf
NULL
1216_統一_2018-03-30_中信投顧_中信個股報告-20180330_1216統一.pdf
NULL
1216_統一_2018-12-26_中信投顧_中信個股報告-20181225_1216統一.pdf
NULL
1216_統一_2019-03-29_中信投顧_中信個股報告-20190329_1216統一.pdf
NULL
1218_泰山_2019-01-02_中信投顧_中信個股報告-20190102_1218泰山.pdf
NULL
1229_聯華_2018-12-03_中信投顧_中信個股報告-2

In [7]:
result.to_csv('result.csv', encoding="utf_8_sig")

In [26]:
root = '../history_data/file/富邦/'
count_false_rating = 0
count_true_rating = 0
count_false_tp = 0
count_true_tp = 0
count_false_author = 0
count_true_arthor = 0
count_false_summary = 0
count_true_summary = 0

result = pd.DataFrame(columns=['filename', 'advisor', 'rating_1', 'rating_2', 'rating', \
                               'tp_1', 'tp_2', 'tp', 'author_1', 'author_2', 'author', \
                                'summary_1', 'summary_2', 'summary'])
count = 0

for filename in os.listdir(root):
    file_path = os.path.join(root, filename)
    pdfReader = Fubon(file_path)
    rating = pdfReader.get_rating_process()
    target_price = pdfReader.get_target_price_process()
    author = pdfReader.get_author_process()
    summary = pdfReader.get_summary_process()

    if rating == 'NULL':
        # print(filename)
        # print(rating)
        # print(target_price)
        # print(author)
        # print(summary)
        count_false_rating+=1
        
    if target_price == 'NULL':
        count_false_tp+=1

    if author == 'NULL':
        count_false_author+=1

    if summary == 'NULL':
        count_false_summary+=1

    new_row = {'filename':filename, 'advisor':pdfReader.advisor, 'rating_1':pdfReader.rating_1, 'rating_2':pdfReader.author_2, 'rating':rating, \
               'tp_1':pdfReader.tp_1, 'tp_2':pdfReader.tp_2, 'tp':target_price, \
                'author_1':pdfReader.author_1, 'author_2':pdfReader.author_2, 'author':author, \
                'summary_1':pdfReader.summary_1, 'summary_2':pdfReader.summary_2, 'summary':summary}
    result = pd.concat([result, pd.DataFrame(new_row, index=[0])], ignore_index=True)
    count += 1
    if count>100:
        break
        
if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_rating
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_rating}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_tp
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_tp}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_author
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_author}件, 成功率:{success_rate}%')

if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false_summary
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false_summary}件, 成功率:{success_rate}%')


../history_data/file/富邦/, 共717件, 成功717件, 失敗0件, 成功率:100.0%
../history_data/file/富邦/, 共717件, 成功685件, 失敗32件, 成功率:95.5%
../history_data/file/富邦/, 共717件, 成功621件, 失敗96件, 成功率:86.6%
../history_data/file/富邦/, 共717件, 成功711件, 失敗6件, 成功率:99.2%


In [2]:
import fitz

In [12]:
doc = fitz.open(r'sample_file\1312_國喬_富邦_中立.pdf')


In [13]:
type(doc) == fitz.fitz.Document

True

In [28]:
display(result)

,filename,advisor,rating_1,rating_2,rating,tp_1,tp_2,tp,author_1,author_2,author,summary_1,summary_2,summary
0,1101_台泥_2018-10-03_富邦台灣_1101_台泥_snapshot_c_100...,Fubon,增加持股,,增加持股,50.0,NULL,50.0,請參閱末頁之免責宣言,,NULL,產業進入旺季,產業進入旺季,產業進入旺季
1,1101_台泥_2019-03-15_富邦台灣_1101_台泥_snapshot_c_031...,Fubon,增加持股,,增加持股,44.3,NULL,44.3,請參閱末頁之免責宣言,,NULL,走出淡季 具高殖利率,走出淡季 具高殖利率,走出淡季 具高殖利率
2,1102_亞泥_2019-03-19_富邦台灣_1102_亞泥_snapshot_c_031...,Fubon,增加持股,,增加持股,43.60,NULL,43.60,請參閱末頁之免責宣言,,NULL,淡季尾聲,淡季尾聲,淡季尾聲
3,1102_亞泥_2019-03-21_富邦台灣_1102_亞泥_snapshot_c_031...,Fubon,增加持股,,增加持股,43.60,NULL,43.60,請參閱末頁之免責宣言,,NULL,淡季尾聲,淡季尾聲,淡季尾聲
4,1102_亞泥_2019-05-16_富邦台灣_1102_亞泥_snapshot_c_051...,Fubon,增加持股,,增加持股,47.5,NULL,47.5,請參閱末頁之免責宣言,,NULL,1Q19 財報佳 全年獲利仍高增長,1Q19 財報佳 全年獲利仍高增長,1Q19 財報佳 全年獲利仍高增長
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2301_光寶科_2018-10-24_富邦台灣_2301_光寶科_snapshot_c_1...,Fubon,增加持股,(886-2) 6606-668,增加持股,40.0,40.0,40.0,(886,(886-2) 6606-668,NULL,高殖利率概念股,高殖利率概念股,高殖利率概念股
97,2301_光寶科_2018-11-03_富邦台灣_2301_光寶科_snapshot_c_1...,Fubon,增加持股,yunting.huang@fub,增加持股,41.00,NULL,41.00,(886 2) 6606 6687,yunting.huang@fub,NULL,3Q18 毛利率表現亮眼,3Q18 毛利率表現亮眼,3Q18 毛利率表現亮眼
98,2301_光寶科_2019-01-17_富邦台灣_2301_光寶科_snapshot_c_0...,Fubon,增加持股,(886-2) 6606-668,增加持股,49.00,NULL,49.00,(886,(886-2) 6606-668,NULL,毛利率穩步走升,毛利率穩步走升,毛利率穩步走升
99,2301_光寶科_2019-01-26_富邦台灣_2301_光寶科_snapshot_c_0...,Fubon,增加持股,(886-2) 6606-668,增加持股,49.00,NULL,49.00,(886,(886-2) 6606-668,NULL,毛利率穩步走升,毛利率穩步走升,毛利率穩步走升


In [ ]:
root = '../history_data/file/國票/'
count_false = 0
count_true = 0
for filename in os.listdir(root):
    file_path = os.path.join(root, filename)

    pdfReader = Ibf(file_path)
    rating = pdfReader.get_rating_process()
    target_price = pdfReader.get_target_price_process()
    author = pdfReader.get_author_process()
    summary = pdfReader.get_summary_process()

    if summary == 'NULL':
        print(filename)
        print(rating)
        print(target_price)
        print(author)
        print(summary)
        count_false+=1
        
        
if len(os.listdir(root))>0:
    count_true = len(os.listdir(root)) - count_false
    success_rate = round((count_true/(len(os.listdir(root)))), 3)*100
    print(f'{root}, 共{len(os.listdir(root))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')


In [ ]:
# specify the path to the compressed file
zip_path = 'file'

# specify the path to the directory where the extracted files will be saved
extract_root = r'output\extract'
processed_root = r'output\processed'
if not os.path.exists(extract_root):
    os.makedirs(extract_root)
if not os.path.exists(processed_root):
    os.makedirs(processed_root) 
# create a ZipFile object and extract the contents to the output directory
for directory in os.listdir(zip_path):
    directory_path = os.path.join(zip_path, directory)
    extract_path = os.path.join(extract_root, os.path.splitext(directory)[0])
    with zipfile.ZipFile(directory_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)


In [ ]:
all_pdf_root = r'../history_data/file'

In [ ]:
for date_path in os.listdir(all_pdf_root):
    directory_date = os.path.join(all_pdf_root, date_path)
    handler_find_advisor(directory_date)
        

In [ ]:
for date_path in os.listdir(extract_root):
    directory_date = os.path.join(extract_root, date_path)
    processed_date = os.path.join(processed_root, date_path)
    date = date_path
    # print(date)
    for directory in os.listdir(directory_date):
        directory_path = os.path.join(directory_date, directory)
        # print(directory_path)zz
        processed_path = os.path.join(processed_date, directory)
        if not os.path.exists(processed_path):
            os.makedirs(processed_path) 
        if directory == 'test':
            handler_find_advisor(directory_path)
        # if directory == '2':
        #     handler_2(directory_path, date, processed_path)
        